# RNN - longseq

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
# window를 한칸씩 옮겨가면서 input으로 넣고 output은 다음 input

In [4]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [5]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10 # 다른 숫자여도 ㄱㄴ
learning_rate=0.1

In [6]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # np.eye => one hot vector

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [7]:
# transform as torch tensor varialbe
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [11]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 그 모듈이 내부적으로 어떤 하위 모듈을 쓰는지 => 여기서는 RNN과 fc
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x): # 정확히 X라는 input을 넣고 network를 어떻게 계산할지
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2) # 2 : num_layers 에 들어가는 layer가 2라서

In [12]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [15]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # model의 output과 True label 비교하여 loss 구함
    loss.backward()
    optimizer.step()

    # 모델이 예측한 결과물 해석
    results = outputs.argmax(dim=2) # argmax를 통해 prediction 확률이 가장 큰 결과값을 구해옴
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result]) # 맨 처음에는 전부 받아옴
        else:
            predict_str += char_set[result[-1]] # 그 이후부터는 맨 마지막 값만 더해줌

    print(predict_str)

l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them task

l sou want to build anship, don't arum up people together to lollect wood and don't assign the  tosks and work, but rather toach the  to long for the endless immensity of the eeas
l sou wont to build a ship, don't arum up people together to collect wood and don't assign the  tosks and work, but rather toach the  to long for the endless immensity of the eeas
l you want to build a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather toach the  to long for the endless immensity of the eeas
m wou want to build a ship, don't drum up people together to lollect wood and don't assign them tosks and work, but rather toach them to long for themendless immensity of themeea.
m wor want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather toach them to cong for themendless immensity of themeea.
m you want to build a ship, don't drum up people together to collect wood and don't dssign them tosk